In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
import xlrd
import csv
from datetime import datetime

In [2]:
# read .csv file
df = pd.read_csv('PI Analyst Data Set Feb 2020.csv')

In [3]:
# visually assess the data set
df.head()

,STORE,CUSTOMER ID,DATE,DAY OF THE WEEK,SHOPPING TRIP ID,UPC,TOTAL AMOUNT(Unit price* QTY),QUANTITY,CATEGORY,SUBCATEGORY,PRODUCT DESCRIPTION,BRAND
0,A,RGWZGML,12/20/2019,Fri,SNVISCLSTBQQ1B5QBX,2.129990e+11,3645.00,9.0,major misc,major misc,(null),(null)
1,C,XX77GXLZ,12/20/2019,Fri,SNVISCLSTBQY1N1TQ3,2.559360e+10,494.67,33.0,Self Service Deli Food Bar,Hot Entrees,DELI MEAL DEAL,(null)
2,C,1MG1MGG,12/11/2019,Wed,SNVISCLSTGXT1YNYYY,8.289611e+09,455.76,24.0,Wine,1.5 L,FETZER VALLEY OAKS CHARDONNAY,FETZER
3,A,XX7RGM1L,12/31/2019,Tue,SNVISCLSTTXY3N33YB,9.999110e+05,434.74,12.0,major misc,major misc,(null),(null)
4,C,RWL7MW7,12/19/2019,Thur,SNVISCLSTBTX215N1N,2.111930e+11,392.33,1.0,major misc,major misc,(null),(null)


In [4]:
# size of data set
print(df.shape)

(988069, 12)


In [5]:
# check data types of each column
df.dtypes

STORE                             object
CUSTOMER ID                       object
DATE                              object
DAY OF THE WEEK                   object
SHOPPING TRIP ID                  object
UPC                              float64
TOTAL AMOUNT(Unit price* QTY)    float64
QUANTITY                         float64
CATEGORY                          object
SUBCATEGORY                       object
PRODUCT DESCRIPTION               object
BRAND                             object
dtype: object

In [6]:
# describe data set to further analyze the data
df.describe()

,UPC,TOTAL AMOUNT(Unit price* QTY),QUANTITY
count,9.880690e+05,988069.000000,988069.000000
mean,2.968028e+10,5.310116,1.225753
std,5.988818e+10,102.151214,0.840279
min,4.000000e+00,-101100.000000,-20.000000
25%,2.840059e+09,2.550000,1.000000
50%,7.056089e+09,3.990000,1.000000
75%,3.006779e+10,6.450000,1.000000
max,9.781980e+11,3645.000000,79.000000


## Category Analysis on per-week basis
- Total revenue ranked highest to lowest
Total revenue for each category/days*7
- Average sales per unit
- Total units moved

In [7]:
# make a copy of data set
df_CA = df.copy()
df_brands = df.copy()

In [8]:
df_CA.head()

,STORE,CUSTOMER ID,DATE,DAY OF THE WEEK,SHOPPING TRIP ID,UPC,TOTAL AMOUNT(Unit price* QTY),QUANTITY,CATEGORY,SUBCATEGORY,PRODUCT DESCRIPTION,BRAND
0,A,RGWZGML,12/20/2019,Fri,SNVISCLSTBQQ1B5QBX,2.129990e+11,3645.00,9.0,major misc,major misc,(null),(null)
1,C,XX77GXLZ,12/20/2019,Fri,SNVISCLSTBQY1N1TQ3,2.559360e+10,494.67,33.0,Self Service Deli Food Bar,Hot Entrees,DELI MEAL DEAL,(null)
2,C,1MG1MGG,12/11/2019,Wed,SNVISCLSTGXT1YNYYY,8.289611e+09,455.76,24.0,Wine,1.5 L,FETZER VALLEY OAKS CHARDONNAY,FETZER
3,A,XX7RGM1L,12/31/2019,Tue,SNVISCLSTTXY3N33YB,9.999110e+05,434.74,12.0,major misc,major misc,(null),(null)
4,C,RWL7MW7,12/19/2019,Thur,SNVISCLSTBTX215N1N,2.111930e+11,392.33,1.0,major misc,major misc,(null),(null)


In [9]:
df_CA.CATEGORY.unique()

array(['major misc', 'Self Service Deli Food Bar', 'Wine', 'Fresh Beef',
       'Fresh Pork', 'Fresh Prepared Meat', 'Bouquets and Cut Flowers',
       'Fresh Shellfish', '(null)', 'Baby Food', 'Bottled Water',
       'Frozen Shellfish', 'Grapes', 'Fresh Fish', 'Produce Juice',
       'Seasonal Merchandise and Misc GM', 'Beer', 'Juice',
       'Self Service Deli Cold', 'Alternative Beverages', 'Housewares',
       'Tobacco', 'Ham', 'Misc Produce', 'Baking Ingredients', 'Bacon',
       'Fresh Chicken', 'Service Deli', 'Fresh Cut', 'In Store Bakery',
       'Frozen Seafood', 'Frozen Snack', 'Frozen Fish',
       'Other Prepared and Ready To Heat', 'Bakery Bread', 'Incontinence',
       'Frozen Poultry and Meat', 'Functional Beverages', 'Other Seafood',
       'Adult Nutrition', 'Vitamins and Supplements', 'Plants',
       'Ice Cream', 'Cat Food', 'Carbonated Beverages', 'Fresh Turkey',
       'Candy', 'Canned Beans', 'Fresh Lamb', 'Dog Food', 'Batteries',
       'Sweet Goods', 'Oil', 'Br

In [10]:
# convert DATE column string to datetime
df_CA['DATE'] = pd.to_datetime(df_CA['DATE'])
df_CA['Week_Number'] = df_CA['DATE'].dt.week
df_sales = df_CA.copy()
df_CA.head()

,STORE,CUSTOMER ID,DATE,DAY OF THE WEEK,SHOPPING TRIP ID,UPC,TOTAL AMOUNT(Unit price* QTY),QUANTITY,CATEGORY,SUBCATEGORY,PRODUCT DESCRIPTION,BRAND,Week_Number
0,A,RGWZGML,2019-12-20,Fri,SNVISCLSTBQQ1B5QBX,2.129990e+11,3645.00,9.0,major misc,major misc,(null),(null),51
1,C,XX77GXLZ,2019-12-20,Fri,SNVISCLSTBQY1N1TQ3,2.559360e+10,494.67,33.0,Self Service Deli Food Bar,Hot Entrees,DELI MEAL DEAL,(null),51
2,C,1MG1MGG,2019-12-11,Wed,SNVISCLSTGXT1YNYYY,8.289611e+09,455.76,24.0,Wine,1.5 L,FETZER VALLEY OAKS CHARDONNAY,FETZER,50
3,A,XX7RGM1L,2019-12-31,Tue,SNVISCLSTTXY3N33YB,9.999110e+05,434.74,12.0,major misc,major misc,(null),(null),1
4,C,RWL7MW7,2019-12-19,Thur,SNVISCLSTBTX215N1N,2.111930e+11,392.33,1.0,major misc,major misc,(null),(null),51


In [11]:
# drop (null) entires
df_CA['CATEGORY'] = df_CA['CATEGORY'].replace('(null)', 'No category listed')

In [12]:
# sum revenue by category
df_CA=df_CA.groupby(['CATEGORY'], as_index=False)['TOTAL AMOUNT(Unit price* QTY)', 'QUANTITY'].sum()

In [13]:
# rename column
df_CA.rename(columns={'TOTAL AMOUNT(Unit price* QTY)':'REVENUE', 'QUANTITY':'UNITS_MOVED'}, inplace=True)
df_CA.head()

,CATEGORY,REVENUE,UNITS_MOVED
0,Adult Nutrition,23995.04,5578.0
1,Air Fresheners,5243.48,1190.0
2,Alternative Beverages,20579.08,4940.0
3,Analgesics,9762.63,1441.0
4,Antacids and Laxatives,7124.40,868.0


In [14]:
df_CA[['AVERAGE_SALES_PER_UNIT']] = df_CA[['REVENUE']].div(df_CA['UNITS_MOVED'].values,axis=0)

In [15]:
month_to_week = 7/31

In [16]:
df_CA[['REVENUE']] = df_CA[['REVENUE']].multiply(month_to_week)
df_CA[['UNITS_MOVED']] = df_CA[['UNITS_MOVED']].multiply(month_to_week)

RuntimeError: module compiled against API version 0xc but this version of numpy is 0xa

In [17]:
df_CA.head()

,CATEGORY,REVENUE,UNITS_MOVED,AVERAGE_SALES_PER_UNIT
0,Adult Nutrition,5418.234839,1259.548387,4.301728
1,Air Fresheners,1184.011613,268.709677,4.406286
2,Alternative Beverages,4646.889032,1115.483871,4.165806
3,Analgesics,2204.464839,325.387097,6.774899
4,Antacids and Laxatives,1608.735484,196.000000,8.207834


In [18]:
# sort by revenue descending
df_CA.sort_values(['REVENUE'], inplace=True, ascending=[False])

In [19]:
# format Revenue column
df_CA['REVENUE'] = df_CA['REVENUE'].map('${:,.2f}'.format)
df_CA['UNITS_MOVED'] = df_CA['UNITS_MOVED'].map('{:,.2f}'.format)
df_CA['AVERAGE_SALES_PER_UNIT'] = df_CA['AVERAGE_SALES_PER_UNIT'].map('${:,.2f}'.format)

In [20]:
# check
df_CA = df_CA.reset_index()
df_CA.head()

,index,CATEGORY,REVENUE,UNITS_MOVED,AVERAGE_SALES_PER_UNIT
0,227,Wine,"$81,832.62","5,066.19",$16.15
1,88,Fresh Beef,"$46,214.17","3,293.84",$14.03
2,20,Beer,"$45,912.61","3,973.97",$11.55
3,197,Self Service Deli Cold,"$42,238.13","7,334.58",$5.76
4,161,No category listed,"$41,014.06","8,147.56",$5.03


In [21]:
# Write object to a comma-separated values (csv) file
df_CA.to_csv('Category Analysis.csv')

In [ ]:
df_weekly sales

## Important Brands

In [22]:
df_brands

,STORE,CUSTOMER ID,DATE,DAY OF THE WEEK,SHOPPING TRIP ID,UPC,TOTAL AMOUNT(Unit price* QTY),QUANTITY,CATEGORY,SUBCATEGORY,PRODUCT DESCRIPTION,BRAND
0,A,RGWZGML,12/20/2019,Fri,SNVISCLSTBQQ1B5QBX,2.129990e+11,3645.00,9.0,major misc,major misc,(null),(null)
1,C,XX77GXLZ,12/20/2019,Fri,SNVISCLSTBQY1N1TQ3,2.559360e+10,494.67,33.0,Self Service Deli Food Bar,Hot Entrees,DELI MEAL DEAL,(null)
2,C,1MG1MGG,12/11/2019,Wed,SNVISCLSTGXT1YNYYY,8.289611e+09,455.76,24.0,Wine,1.5 L,FETZER VALLEY OAKS CHARDONNAY,FETZER
3,A,XX7RGM1L,12/31/2019,Tue,SNVISCLSTTXY3N33YB,9.999110e+05,434.74,12.0,major misc,major misc,(null),(null)
4,C,RWL7MW7,12/19/2019,Thur,SNVISCLSTBTX215N1N,2.111930e+11,392.33,1.0,major misc,major misc,(null),(null)
5,C,XXLRZLZL,12/21/2019,Sat,SNVISCLSTBYQBTBTX2,2.112350e+11,390.08,1.0,Fresh Beef,Steak,DRY AGED OVEN READY BONE IN RIB ROAST,(null)
6,A,RL7RMW4,12/21/2019,Sat,SNVISCLSTB2YTT1N55,2.111960e+11,326.52,1.0,Fresh Beef,Roast,Beef Oven Ready Rib Roast,ALL OTHER BRANDS
7,C,RL7RMW4,12/27/2019,Fri,SNVISCLST3QXX255NT,2.131820e+11,296.87,15.0,Fresh Pork,Roast,HORMEL NATURAL CHOICE PORK BUTTS,HORMEL
8,A,MZRMW4W,12/23/2019,Mon,SNVISCLST3XQX3Y511,2.111960e+11,288.32,1.0,Fresh Beef,Roast,Beef Oven Ready Rib Roast,ALL OTHER BRANDS
9,A,XXGMM4LX,12/7/2019,Sat,SNVISCLSTX2YTQ3T5Q,2.137830e+11,283.05,10.0,Fresh Prepared Meat,Fresh Prepared Meat,SMOKEHOUSE SPIRAL W/BROWN SUGAR,ALL OTHER BRANDS


In [23]:
# sum revenue by brand
df_brands=df_brands.groupby(['BRAND'], as_index=False)['TOTAL AMOUNT(Unit price* QTY)', 'QUANTITY'].sum()

In [24]:
# rename (null) entires
df_brands['BRAND'] = df_brands['BRAND'].replace('(null)', 'NO BRAND LISTED')
df_brands.head()

,BRAND,TOTAL AMOUNT(Unit price* QTY),QUANTITY
0,NO BRAND LISTED,601919.71,98698.63
1,100 GRAND,156.20,142.00
2,1000 STORIES,307.86,14.00
3,123,20.34,6.00
4,14 HANDS,2453.33,167.00


In [25]:
# rename column
df_brands.rename(columns={'TOTAL AMOUNT(Unit price* QTY)':'REVENUE', 'QUANTITY':'UNITS_MOVED'}, inplace=True)
df_brands.head()

,BRAND,REVENUE,UNITS_MOVED
0,NO BRAND LISTED,601919.71,98698.63
1,100 GRAND,156.20,142.00
2,1000 STORIES,307.86,14.00
3,123,20.34,6.00
4,14 HANDS,2453.33,167.00


In [26]:
# sort by revenue descending
df_brands.sort_values(['REVENUE'], inplace=True, ascending=[False])

In [27]:
df_brands.head()

,BRAND,REVENUE,UNITS_MOVED
83,ALL OTHER BRANDS,748636.60,194006.99
0,NO BRAND LISTED,601919.71,98698.63
2703,PRIVATE LABEL,446660.74,144065.00
383,BOARS HEAD,76011.56,10083.00
1257,FRESH EXPRESS,38802.00,9940.00


In [28]:
df_brands[['AVERAGE_SALES_PER_UNIT']] = df_brands[['REVENUE']].div(df_brands['UNITS_MOVED'].values,axis=0)

In [29]:
df_brands['REVENUE'] = df_brands['REVENUE'].map('${:,.2f}'.format)
df_brands['UNITS_MOVED'] = df_brands['UNITS_MOVED'].map('{:,.2f}'.format)

In [31]:
df_brands.tail(10)

,BRAND,REVENUE,UNITS_MOVED,AVERAGE_SALES_PER_UNIT
3621,VOSS,$1.59,1.00,1.590000
2203,MOJO,$1.49,1.00,1.490000
1085,EL PATO,$1.45,1.00,1.450000
2717,PUDDLE WINKS,$1.00,0.00,inf
726,CLARISSE,$1.00,1.00,1.000000
1329,GENERAL MILLS CHEX MIX,$0.00,0.00,NaN
3257,STREITS,$0.00,0.00,NaN
1639,ITÃÂS A 10,$0.00,0.00,NaN
1345,GIFTCARD,"$-1,275.00",-54.00,23.611111
1344,GIFT CARD,"$-190,562.33","-2,244.00",84.920824


In [578]:
# Write object to a comma-separated values (csv) file
df_brands.to_csv('Important Brands.csv')

## Best Performing Store

In [580]:
df_store = df.copy()

In [581]:
# sum revenue by store
df_store=df_store.groupby(['STORE'], as_index=False)['TOTAL AMOUNT(Unit price* QTY)', 'QUANTITY'].sum()

In [582]:
# rename column
df_store.rename(columns={'TOTAL AMOUNT(Unit price* QTY)':'REVENUE', 'QUANTITY':'UNITS_MOVED'}, inplace=True)
df_store.head()

,STORE,REVENUE,UNITS_MOVED
0,A,2.192678e+06,494964.99
1,B,9.880333e+05,229585.92
2,C,1.619924e+06,370912.45
3,D,4.461270e+05,115665.53


In [583]:
# sort by revenue descending
df_store.sort_values(['REVENUE'], inplace=True, ascending=[False])

In [585]:
df_store.head()

,STORE,REVENUE,UNITS_MOVED
0,A,2.192678e+06,494964.99
2,C,1.619924e+06,370912.45
1,B,9.880333e+05,229585.92
3,D,4.461270e+05,115665.53


In [587]:
df_store['REVENUE'] = df_store['REVENUE'].map('${:,.2f}'.format)
df_store['UNITS_MOVED'] = df_store['UNITS_MOVED'].map('{:,.2f}'.format)

In [588]:
df_store.head()

,STORE,REVENUE,UNITS_MOVED
0,A,"$2,192,677.67","494,964.99"
2,C,"$1,619,923.52","370,912.45"
1,B,"$988,033.32","229,585.92"
3,D,"$446,126.96","115,665.53"


In [589]:
# Write object to a comma-separated values (csv) file
df_store.to_csv('Best Performing Store.csv')